# S-Parameter Analysis for 2-Port components
Author: Kwok Keith

Date: 9 Jan 2026

In [5]:
import numpy as np
from dataclasses import dataclass
from typing import Optional
import json, csv
from pathlib import Path

In [80]:
@dataclass(frozen=True)
class Component:
    name: str
    type: str
    S_parameter: dict[np.int64, np.ndarray[np.complex128]] # 2 x 2 S-parameter matrix at diff freq

class Network:
    _components: list[Component]
    z_0: float = 50.0    # Characteristic impedance in Ohms
    S_matrix: dict[np.int64, np.ndarray[np.complex128]] # Overall S-matrix at different frequencies
    freqs: list[np.int64] = None # Frequencies at which S-parameters are defined

    def __init__(self, components: list[Component], freqs: list[np.int64] = [], z_0: float = 50.0) -> None:
        self._components = components
        self.z_0 = z_0
        self.freqs = freqs
        self._update_Sparameters()

    def _update_Sparameters(self) -> None:
        """Updates the overall S-parameters of the network."""
        self.S_matrix = {} # Reset S_matrix
        for freq in self.freqs:
            S_11, S_12, S_21, S_22 = self.overall_sparameters(freq=freq)
            self.S_matrix[freq] = np.array([[S_11, S_12], [S_21, S_22]], dtype=complex)


    def insert_component(self, component: Component, idx: Optional[int]) -> None:
        """Inserts a component into the network, if idx is given then at the position, otherwise at the end of the network."""
        if idx is not None:
            if idx < 0 or idx > len(self._components):
                raise IndexError(f"Index {idx} is out of range for components list.")
            self._components.insert(idx, component)
        else:
            self._components.append(component)
        self._update_Sparameters()


    def remove_component(self, position_idx: int) -> None:
        """Removes a component from the network based on its position index."""
        if position_idx < 0 or position_idx >= len(self._components):
            raise IndexError(f"Position index {position_idx} is out of range for components list.")
        self._components.pop(position_idx)
        self._update_Sparameters()


    def export_s_matrix(self) -> dict[np.int64, np.ndarray[np.complex128]]:
        """Exports the overall S-matrix of the network as a 2x2 numpy array."""
        return self.S_matrix


    def display_network(self, freqs: Optional[list[np.int64]]=None) -> None:
        """Outputs the network components and their overall S-parameters."""
        print("Network Components:")
        if freqs is not None:
            for freq in freqs:
                if freq not in self.freqs:
                    raise ValueError(f"Frequency at {freq} not within the frequencies of the Network.")
                print(f"Frequency: {freq} Hz")
                for idx, comp in enumerate(self._components):
                    print(f"  Component {idx}: {comp.name}, type = {comp.type} ")
                    print(f"  S-Parameters: {comp.S_parameter[freq]}\n")
        else:
            for idx, comp in enumerate(self._components):
                print(f"  Component {idx}: {comp.name}, type = {comp.type} ")


    @staticmethod
    def s_to_abcd(self, 
            S_11 : complex,
            S_12 : complex, 
            S_21 : complex, 
            S_22 : complex) -> tuple[complex, complex, complex, complex]:
        """Converts S-parameters to ABCD parameters."""
        A = ((1 + S_11) * (1 - S_22) + S_12 * S_21) / (2 * S_21)
        B = ((1 + S_11) * (1 + S_22) - S_12 * S_21) / (2 * S_21) * self.z_0
        C = ((1 - S_11) * (1 - S_22) - S_12 * S_21) / (2 * S_21) / self.z_0
        D = ((1 - S_11) * (1 + S_22) + S_12 * S_21) / (2 * S_21)
        return A, B, C, D

    @staticmethod
    def abcd_to_s(self, 
            A : complex, 
            B : complex, 
            C : complex, 
            D : complex) -> tuple[complex, complex, complex, complex]:
        """Converts ABCD-parameters to S parameters."""
        denom = (A + B / self.z_0 + C * self.z_0 + D)
        S_11 = (A + B / self.z_0 - C * self.z_0 - D) / denom
        S_12 = 2 * (A * D - B * C) / denom
        S_21 = 2 / denom
        S_22 = (-A + B / self.z_0 - C * self.z_0 + D) / denom
        return S_11, S_12, S_21, S_22

    def overall_sparameters(self, freq : np.int64) -> tuple[complex, complex, complex, complex]:
        """Calculates the overall S-parameters of the network by cascading individual component S-parameters."""
        # Handle empty component list
        if not self._components:
            return 0.0, 0.0, 0.0, 0.0

        total_abcd_matrix = np.array([[1, 0], [0, 1]], dtype=complex)

        # Convert each component S-param to ABCD and cascade
        for comp in self._components:
            comp_S11, comp_S12, comp_S21, comp_S22 = comp.S_parameter[freq].flatten()
            A, B, C, D = self.s_to_abcd(self, comp_S11, comp_S12, comp_S21, comp_S22)
            total_abcd_matrix = total_abcd_matrix @ np.array([[A, B], [C, D]], dtype=complex)
        
        A, B, C, D = total_abcd_matrix.flatten()
        return self.abcd_to_s(self, A, B, C, D) # Convert ABCD back to S-parameters

In [81]:
try:
    base = Path(__file__).parent
except NameError:
    base = Path.cwd()

path_to_network_components = base / "data" / "S_param_components.json"
path_to_S_parameters_fdr = base / "data" / "S_parameters"

freqs: list[float] = [1e9]  # Frequencies in Hz

def load_component(comp_data: dict, freqs: list[float], tol_hz: float = 1.0) -> Component:
    S_param_dict: dict[float, np.ndarray] = {}
    S_param_file = path_to_S_parameters_fdr / f"{comp_data['type']}.csv"

    with open(S_param_file, "r", newline="") as csvfile:
        reader = csv.DictReader(csvfile)
        required = {"Hz", "S11_A", "S11_p", "S21_A", "S21_p", "S12_A", "S12_p", "S22_A", "S22_p"}
        if not required.issubset(set(reader.fieldnames or [])):
            raise ValueError(f"{S_param_file} must contain columns: {', '.join(sorted(required))}")
        
        # Function converts amplitude and degree phase into complex representation
        form_complex = lambda S_A, S_p : \
            complex(float(row[S_A]) * np.cos(np.deg2rad(float(row[S_p]))), 
                    float(row[S_A]) * np.sin(np.deg2rad(float(row[S_p]))))

        for row in reader:
            f = np.int64(row["Hz"])
            # Match only requested freqs (within tolerance)
            for tf in freqs:
                if np.isclose(f, tf, rtol=0.0, atol=tol_hz):
                    S11 = form_complex("S11_A", "S11_p")
                    S12 = form_complex("S12_A", "S12_p")
                    S21 = form_complex("S21_A", "S21_p")
                    S22 = form_complex("S22_A", "S21_p")
                    S_param_dict[tf] = np.array([[S11, S12], [S21, S22]], dtype=np.complex128)
                    break

    # Output warning if any requested freqs are missing
    missing = [tf for tf in freqs if tf not in S_param_dict]
    if missing:
        print(f"Warning: {S_param_file.name} missing rows for freqs: {missing}")

    return Component(name=comp_data["name"], type=comp_data["type"], S_parameter=S_param_dict)

# Build components using only the matched frequency rows
components: list[Component] = []
with open(path_to_network_components, "r") as f:
    components_data = json.load(f)
    for comp_data in components_data:
        components.append(load_component(comp_data, freqs))

In [82]:
network = Network(components=components, freqs=freqs)
network.display_network(freqs=[1e9])

Network Components:
Frequency: 1000000000.0 Hz
  Component 0: LFCG-1200+_0, type = LFCG-1200_Plus25degC 
  S-Parameters: [[10.62349438-27.01533187j  0.54103465 +0.26851308j]
 [ 0.53735946 +0.26660227j 28.61231099+14.1955385j ]]

  Component 1: EHA-163L+_0, type = EHA-163L_Plus25degC 
  S-Parameters: [[ 5.49836613+13.97887426j  8.68617489+17.03978409j]
 [ 9.10351961+11.94549244j -8.00348056-10.50203884j]]



In [83]:
network.export_s_matrix()

{1000000000.0: array([[ 1.06133248e+01-2.70203527e+01j, -2.39563821e-02+2.37036014e-03j],
        [-1.80508028e-02+5.16172167e-03j, -2.10353136e+01-2.44922549e+01j]])}